In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn 
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from deduping_module import deduping_class

In [2]:
link = r'C:\Users\USER\Documents\LM_project\SAL-230_Revisit validity'

In [3]:
gt = pd.read_csv(link + '\\raw\sf_export.csv', encoding='latin-1')
nm = pd.read_csv(link + '\\raw\pennsylvania-00001.csv', encoding='latin-1')

In [4]:
test = dm.deduping_class(gt) 

NameError: name 'dm' is not defined

In [ ]:
test.ground_truth

In [ ]:
gt.dropna(inplace= True)

In [ ]:
key_list = ['Account Name', 'Phone']

In [ ]:
gt = pd.read_csv('raw/sf_export.csv')
gt.dropna(subset=['Salesforce Contact Id'], inplace= True)
gt.drop_duplicates(subset=['Salesforce Account Id'], inplace= True)
gt['Key Value'] = gt.apply(lambda x: (str(x['Account Name'])+ ' ' + str(x['State 1'])).lower(), axis= 1)
gt.reset_index(drop=True, inplace=True)

In [ ]:
nm = pd.read_csv('raw/pennsylvania-00001.csv')

In [ ]:
nm['Key Value'] = nm.apply(lambda x: (str(x['firstName']) + ' ' + str(x['lastName']) + ' ' + str('PA')).lower(), axis= 1)

In [ ]:
import re

def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
combined_list = nm['Key Value'].tolist() + gt['Key Value'].tolist()
vectorizer.fit(nm['Key Value'].tolist() + gt['Key Value'].tolist())

nm_tfidf = nm['Key Value'].tolist()
nm_tfidf = vectorizer.transform(nm_tfidf)

gt_tfidf = gt['Key Value']
gt_tfidf = vectorizer.transform(gt_tfidf)

In [ ]:
nm_tfidf.shape

In [ ]:
#gt meaning ground truth or reference table

gt_tfidf.shape

In [ ]:
126060 * 1029533

In [ ]:
matches = awesome_cossim_topn(nm_tfidf, gt_tfidf.transpose(), 10, 0.8, use_threads=True, n_jobs=6)

In [ ]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    index_value = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    contact_id = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = nm.loc[sparserows[index], 'Key Value']
        index_value[index] = sparserows[index]
        right_side[index] = gt.loc[sparsecols[index], 'Key Value']
        contact_id[index] = gt.loc[sparsecols[index], 'Salesforce Contact Id']
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'index':index_value,
                          'left_side': left_side,
                          'right_side': right_side,
                          'Contact ID':contact_id, 
                           'similairity': similairity})

In [ ]:
matches_df = get_matches_df(matches, combined_list, top=100000)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
# matches_df.sample(10)


In [ ]:
matches_df.to_csv('result.csv', index=False)

In [ ]:
matches_df.sort_values(by=['similairity'], ascending= False)